## Download/Import Dependencies

In [ ]:
!python --version
# !pip list -v | grep google-colab

Python 3.7.13


In [ ]:
# ### Install these when running for the first time  ######
!pip install contractions
!pip install mistletoe
!pip install html2text

!pip install -U -q PyDrive
!pip install google-colab

import nltk
nltk.download('stopwords')  
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 15.9 MB/s 
     |████████████████████████████████| 287 kB 72.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import re, string, unicodedata    
import contractions   
from bs4 import BeautifulSoup

from mistletoe import markdown
from html2text import HTML2Text

import numpy as np
import pandas as pd 

from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#to import files from google drive
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive
from google.colab import auth 
from oauth2client.client import GoogleCredentials
from google.colab import drive

import difflib
import webbrowser

from datetime import datetime as dt
import time

import copy

In [ ]:
# Runtime GPU info
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [ ]:
# Runtime Memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## Compress and upload the data to  Google Drive and Import the data from there

In [ ]:
# 1. Authorize Google Drive (required only for the first time, then comment it)
drive.mount('/content/gdrive', force_remount=True)

# 2. Read the CSV file from Google Drive saved in the above path
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ClassSpecsBLS.csv', encoding='utf-8')
df.shape

(800807, 14)

In [ ]:
df.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed
0,1,9,AGENCY JOB OPPORTUNITY,This class specification has been entered for ...,Varies by agency and posting.,Varies by agency and posting.,NaN,NaN,All employment offers are contingent upon succ...,"As an Approved Local Merit System, the County ...",Community and Social Service Occupations,"Counselors, Social Workers, and Other Communit...",Miscellaneous Community and Social Service Spe...,Probation Officers and Correctional Treatment ...
1,2,9,ELECTIONS COORDINATOR - SERVICES,"Under direction, to review, research, and anal...","• Reviews, researches, and analyzes elections ...","Knowledge of: Federal, State, and local laws, ...",NaN,Skill: Keyboard skill of 35 words per minute ...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Office and Administrative Support Occupations,Information and Record Clerks,Miscellaneous Information and Record Clerks,"Information and Record Clerks, All Other"
2,3,9,ELECTIONS COORDINATOR'S ASSISTANT,"Under general supervision, to assist an Electi...","• Researches, prepares, and recommends writte...","Knowledge of: Basic Federal, California State...",NaN,Skill: Keyboard skill to type 35 words per mi...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Office and Administrative Support Occupations,Information and Record Clerks,Miscellaneous Information and Record Clerks,"Information and Record Clerks, All Other"
3,4,9,ELECTIONS TECHNICIAN SUPERVISOR,"Under direction, to supervise a clerical funct...","• Supervises, assigns, instructs, and reviews ...",Knowledge of: Principles and techniques of sup...,NaN,Skill: Skill to type 35 words per minute may b...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Office and Administrative Support Occupations,Supervisors of Office and Administrative Suppo...,First-Line Supervisors of Office and Administr...,First-Line Supervisors of Office and Administr...
4,5,9,ELECTIONS TECHNICIAN III - SERVICES,To serve the voting and candidate public by pe...,"• Reviews reports, records, and other documen...",Knowledge of: Modern office practices and pro...,NaN,Skill: Sufficient keyboard or typing skill to...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Office and Administrative Support Occupations,Information and Record Clerks,Miscellaneous Information and Record Clerks,"Information and Record Clerks, All Other"


In [ ]:
df.columns

Index(['ClassSpecID', 'EmployerID', 'Class_Title', 'Class_Concept',
       'ExamplesOfDuties', 'MinimumQualifications', 'SupplementalInfo',
       'OtherRequirements', 'OtherTitle1', 'OtherTitle2', 'BLS_Major',
       'BLS_Minor', 'BLS_Broad', 'BLS_Detailed'],
      dtype='object')

## Subsetting the data for analysis: (keep only nurse practitioners)

In [ ]:
df2 = df[df['BLS_Detailed']=='Nurse Practitioners']
print(df2.shape)
df2.head()

(636, 14)


,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed
589,793,9,MEDICAL CONSULTANT,"Under direction, to provide services for vario...","• Conducts public health clinics, gathers and...",Experience: One year of post-certification ex...,NaN,License/Certificate: Possession of a valid Ph...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners
646,883,9,NURSE PRACTITIONER I,"<div style=""text-align: justify;"">Under medica...","<div style=""text-align: justify;""><br>• Takes ...","<div style=""text-align: justify;"">Experience: ...",NaN,"<div style=""text-align: justify;"">License/Cert...","<div style=""text-align: justify;"">All employme...","<div style=""text-align: justify;"">As an Approv...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners
647,884,9,NURSE PRACTITIONER II,<div>Under medical supervision of a physician ...,<div><br>• Takes patient health histories; per...,<div>Experience: One year of experience as a n...,NaN,<div>License/Certificate: Must possess and mai...,<div>All employment offers are contingent upon...,"<div>As an Approved Local Merit System, all Co...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners
648,885,9,NURSE PRACTITIONER III,<div>Under medical supervision of a physician ...,<div>• Takes patient health histories; perform...,Experience: Three years of experience as a nur...,NaN,License/Certificate: Must possess and maintain...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners
650,888,9,NURSE PRACTITIONER I - PER DIEM,Under medical supervision of a physician and w...,&bull; Takes patient health histories; perform...,Experience: None required. Knowledge of: Tec...,NaN,License/Certificate: Must possess and maintain...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners


In [ ]:
# df2 = df[df['ClassSpecID']==4916]
# df2

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed
3571,4916,141,Family Nurse Practitioner,<strong>Bargaining Unit: CNA - Registered Nurs...,Positions in this class provide in-home or out...,<strong>License Required:</strong> Possession ...,<strong>Knowledge of:</strong><ul>\t<li>The me...,<strong>Under medical direction and consultati...,Established: August 1975<br>Revised: September...,NaN,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners


In [ ]:
# #Remove trailing and leading spaces from strings
# df_obj = df.select_dtypes(['object'])
# df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [ ]:
# #subset some class titles
# df2 = df.loc[df.Class_Title.str.contains("Assistant Caseworker|Community Health Nurse|Nurse Practitioner|Attorney Investigator", na=False)]
# # df2.to_csv("test1.csv")
# print(df2.shape)
# df2.head()
# # type(df.Class_Title[0])

(494, 14)


,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed
2358,3243,168,Assistant Caseworker II,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","<b><u>Knowledge of:</b></u> English usage, sp...",NaN,NaN,NaN,NaN,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides
2360,3245,168,Assistant Caseworker I,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","<b><u>Knowledge of:</b></u> English usage, sp...",NaN,NaN,NaN,NaN,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides
2419,3308,168,Nurse Practitioner,To perform physical screening assessments; to ...,"<b><u>DUTIES</b></u> may include, but are not ...",<b><u>NECESSARY EMPLOYMENT STANDARDS Knowledg...,NaN,NaN,NaN,NaN,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners
3571,4916,141,Family Nurse Practitioner,<strong>Bargaining Unit: CNA - Registered Nurs...,Positions in this class provide in-home or out...,<strong>License Required:</strong> Possession ...,<strong>Knowledge of:</strong><ul>\t<li>The me...,<strong>Under medical direction and consultati...,Established: August 1975<br>Revised: September...,NaN,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners
4394,5875,385,Nurse Practitioner,"Under general supervision, provide health care...","<strong>Duties may include, but are not limite...",<strong>Note: </strong>The level and scope of ...,NaN,NaN,Nurse Practitioner Trainee-Nurse Practitioner,"Established: 10/26/1978<br>Revised: 8/29/2005,...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners


In [ ]:
# sample_titles = ['Assistant Caseworker I','Assistant Caseworker II', 'Assistant Caseworker III',\
#                 #  'Community Health Nurse 3A', 'Community Health Nurse 2A', 'Community Health Nurse 1A',\
#                  'Nurse Practitioner I','Nurse Practitioner II',\
#                  "State's Attorney Investigator I", "State's Attorney Investigator II", "State's Attorney Investigator III"]

# sample_BLS_Detailed = ['Information and Record Clerks, All Other', 'Nurse Practitioners', 'Home Health Aides', 'Lawyers', 'Registered Nurses']
# # df['Class_Title'] = df['Class_Title'].apply(lambda x: str(x).strip())
# df2 = df[df['BLS_Detailed'].isin(sample_BLS_Detailed)]
# print(df2.shape)
# df2.groupby(['BLS_Detailed']).size()
# df2.head()

In [ ]:
# describing stats of variables
df2.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ClassSpecID,636.0,NaN,NaN,NaN,903985.416667,434381.316854,793.0,775267.0,1013553.0,1232997.25,1443422.0
EmployerID,636.0,NaN,NaN,NaN,2527.537736,1935.703225,9.0,1470.0,2051.0,2937.75,11555.0
Class_Title,636,443,Nurse Practitioner,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Class_Concept,593,569,<strong>PRIMARY DUTY: </strong>The Staff Nurse...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ExamplesOfDuties,557,539,<strong>ESSENTIAL FUNCTIONS:</strong><br><ol s...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MinimumQualifications,549,522,<strong>MINIMUM QUALIFICATIONS:</strong><br> ...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SupplementalInfo,472,434,The Maricopa County Human Resources Department...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherRequirements,269,232,<br />,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherTitle1,206,180,<div>All employment offers are contingent upon...,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherTitle2,172,124,"<div>As an Approved Local Merit System, all Co...",10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Count of unique values
# df2.Class_Title.nunique()
df2.ClassSpecID.nunique()

#check for nulls
df2.isnull().sum(axis=0)     

ClassSpecID                0
EmployerID                 0
Class_Title                0
Class_Concept             43
ExamplesOfDuties          79
MinimumQualifications     87
SupplementalInfo         164
OtherRequirements        367
OtherTitle1              430
OtherTitle2              464
BLS_Major                  0
BLS_Minor                  0
BLS_Broad                  0
BLS_Detailed               0
dtype: int64

## Data Pre-processing:

In [ ]:
df2.fillna('', inplace=True)

In [ ]:
df2['New_JD'] = df2['Class_Concept'] + "<br><br>" + df2['ExamplesOfDuties'] + "<br><br>" + df2['MinimumQualifications'] + "<br><br>" + df2['SupplementalInfo']+ "<br><br>" + df2['OtherRequirements'] + "<br><br>" + df2['OtherTitle1'] + "<br><br>" + df2['OtherTitle2']
df2.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,New_JD
589,793,9,MEDICAL CONSULTANT,"Under direction, to provide services for vario...","• Conducts public health clinics, gathers and...",Experience: One year of post-certification ex...,,License/Certificate: Possession of a valid Ph...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"Under direction, to provide services for vario..."
646,883,9,NURSE PRACTITIONER I,"<div style=""text-align: justify;"">Under medica...","<div style=""text-align: justify;""><br>• Takes ...","<div style=""text-align: justify;"">Experience: ...",,"<div style=""text-align: justify;"">License/Cert...","<div style=""text-align: justify;"">All employme...","<div style=""text-align: justify;"">As an Approv...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"<div style=""text-align: justify;"">Under medica..."
647,884,9,NURSE PRACTITIONER II,<div>Under medical supervision of a physician ...,<div><br>• Takes patient health histories; per...,<div>Experience: One year of experience as a n...,,<div>License/Certificate: Must possess and mai...,<div>All employment offers are contingent upon...,"<div>As an Approved Local Merit System, all Co...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,<div>Under medical supervision of a physician ...
648,885,9,NURSE PRACTITIONER III,<div>Under medical supervision of a physician ...,<div>• Takes patient health histories; perform...,Experience: Three years of experience as a nur...,,License/Certificate: Must possess and maintain...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,<div>Under medical supervision of a physician ...
650,888,9,NURSE PRACTITIONER I - PER DIEM,Under medical supervision of a physician and w...,&bull; Takes patient health histories; perform...,Experience: None required. Knowledge of: Tec...,,License/Certificate: Must possess and maintain...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,Under medical supervision of a physician and w...


In [ ]:
data = df2.copy()

#Create Job Description column by merging Class_Concept, ExamplesOfDuties, MinimumQualifications, SupplementalInfo, OtherRequirements columns
data['Job_Desc'] = data[data.columns[3:9]].apply(lambda x: '. '.join(x.dropna().astype(str)), axis=1)

#Remove all the special characters in the text
data['Job_Desc'] = data['Job_Desc'].apply(lambda x: str(x).replace('&nbsp;', '.'))
data['Job_Desc'] = data['Job_Desc'].apply(lambda x: str(x).replace('&rsquo;', ''))
data['Job_Desc'] = data['Job_Desc'].apply(lambda x: str(x).replace('&ldquo;', ''))
data['Job_Desc'] = data['Job_Desc'].apply(lambda x: str(x).replace('&rdquo;', ''))

data['Job_Desc'] = data['Job_Desc'].apply(lambda x: str(x).replace('Â', '.'))
data['Job_Desc'] = data['Job_Desc'].apply(lambda x: str(x).replace('â€™',"'"))

data['Job_Desc'] = data['Job_Desc'].apply(lambda x: str(x).replace('</li>', '.</li>')) #add period before list item ends
data['Job_Desc'] = data['Job_Desc'].apply(lambda x: str(x).replace('<br>', '.<br>')) #add period before line break
data['Job_Desc'] = data['Job_Desc'].apply(lambda x: str(x).replace('</strong>', '.</strong>')) #add period before bold text ends


def strip_html(text):
    # soup = BeautifulSoup(text, "html.parser")
    soup = BeautifulSoup(text, "lxml")
    return soup.get_text()

data['Job_Desc'] = data['Job_Desc'].apply(lambda x: strip_html(str(x)))


# data['Class_Concept'] = data['Class_Concept'].apply(lambda x: strip_html(str(x)))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: strip_html(str(x)))
# data['MinimumQualifications'] = data['MinimumQualifications'].apply(lambda x: strip_html(str(x)))
# data['SupplementalInfo'] = data['SupplementalInfo'].apply(lambda x: strip_html(str(x)))
# data['OtherRequirements'] = data['OtherRequirements'].apply(lambda x: strip_html(str(x)))
# data['OtherTitle1'] = data['OtherTitle1'].apply(lambda x: strip_html(str(x)))
# data['OtherTitle2'] = data['OtherTitle2'].apply(lambda x: strip_html(str(x)))

#Remove all the bullets in the text
# data['Class_Concept'] = data['Class_Concept'].apply(lambda x: str(x).replace('•', ''))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['MinimumQualifications'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['OtherTitle1'] = data['OtherTitle1'].apply(lambda x: str(x).replace('•', ''))

data.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,New_JD,Job_Desc
589,793,9,MEDICAL CONSULTANT,"Under direction, to provide services for vario...","• Conducts public health clinics, gathers and...",Experience: One year of post-certification ex...,,License/Certificate: Possession of a valid Ph...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"Under direction, to provide services for vario...","Under direction, to provide services for vario..."
646,883,9,NURSE PRACTITIONER I,"<div style=""text-align: justify;"">Under medica...","<div style=""text-align: justify;""><br>• Takes ...","<div style=""text-align: justify;"">Experience: ...",,"<div style=""text-align: justify;"">License/Cert...","<div style=""text-align: justify;"">All employme...","<div style=""text-align: justify;"">As an Approv...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"<div style=""text-align: justify;"">Under medica...",Under medical supervision of a physician and w...
647,884,9,NURSE PRACTITIONER II,<div>Under medical supervision of a physician ...,<div><br>• Takes patient health histories; per...,<div>Experience: One year of experience as a n...,,<div>License/Certificate: Must possess and mai...,<div>All employment offers are contingent upon...,"<div>As an Approved Local Merit System, all Co...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,<div>Under medical supervision of a physician ...,Under medical supervision of a physician and w...
648,885,9,NURSE PRACTITIONER III,<div>Under medical supervision of a physician ...,<div>• Takes patient health histories; perform...,Experience: Three years of experience as a nur...,,License/Certificate: Must possess and maintain...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,<div>Under medical supervision of a physician ...,Under medical supervision of a physician and w...
650,888,9,NURSE PRACTITIONER I - PER DIEM,Under medical supervision of a physician and w...,&bull; Takes patient health histories; perform...,Experience: None required. Knowledge of: Tec...,,License/Certificate: Must possess and maintain...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,Under medical supervision of a physician and w...,Under medical supervision of a physician and w...


In [ ]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

data['Job_Desc'] = data['Job_Desc'].apply(lambda x: replace_contractions(x))
data.to_csv("/content/gdrive/My Drive/Colab Notebooks/Nurse_Practitioners_Raw_JD.csv")
data.to_csv('test.csv', index=False)
data.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,New_JD,Job_Desc
589,793,9,MEDICAL CONSULTANT,"Under direction, to provide services for vario...","• Conducts public health clinics, gathers and...",Experience: One year of post-certification ex...,,License/Certificate: Possession of a valid Ph...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"Under direction, to provide services for vario...","Under direction, to provide services for vario..."
646,883,9,NURSE PRACTITIONER I,"<div style=""text-align: justify;"">Under medica...","<div style=""text-align: justify;""><br>• Takes ...","<div style=""text-align: justify;"">Experience: ...",,"<div style=""text-align: justify;"">License/Cert...","<div style=""text-align: justify;"">All employme...","<div style=""text-align: justify;"">As an Approv...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"<div style=""text-align: justify;"">Under medica...",Under medical supervision of a physician and w...
647,884,9,NURSE PRACTITIONER II,<div>Under medical supervision of a physician ...,<div><br>• Takes patient health histories; per...,<div>Experience: One year of experience as a n...,,<div>License/Certificate: Must possess and mai...,<div>All employment offers are contingent upon...,"<div>As an Approved Local Merit System, all Co...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,<div>Under medical supervision of a physician ...,Under medical supervision of a physician and w...
648,885,9,NURSE PRACTITIONER III,<div>Under medical supervision of a physician ...,<div>• Takes patient health histories; perform...,Experience: Three years of experience as a nur...,,License/Certificate: Must possess and maintain...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,<div>Under medical supervision of a physician ...,Under medical supervision of a physician and w...
650,888,9,NURSE PRACTITIONER I - PER DIEM,Under medical supervision of a physician and w...,&bull; Takes patient health histories; perform...,Experience: None required. Knowledge of: Tec...,,License/Certificate: Must possess and maintain...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,Under medical supervision of a physician and w...,Under medical supervision of a physician and w...


# Create Pair-wise Master data

In [ ]:
# Subset the data
data2 = data[['ClassSpecID', 'EmployerID', 'Class_Title', 'New_JD', 'Job_Desc','BLS_Detailed']]
data2.fillna('', inplace=True)
data2.dropna(how='all', axis=1, inplace=True)
print(data2.shape)
data2.head()

(636, 6)


,ClassSpecID,EmployerID,Class_Title,New_JD,Job_Desc,BLS_Detailed
589,793,9,MEDICAL CONSULTANT,"Under direction, to provide services for vario...","Under direction, to provide services for vario...",Nurse Practitioners
646,883,9,NURSE PRACTITIONER I,"<div style=""text-align: justify;"">Under medica...",Under medical supervision of a physician and w...,Nurse Practitioners
647,884,9,NURSE PRACTITIONER II,<div>Under medical supervision of a physician ...,Under medical supervision of a physician and w...,Nurse Practitioners
648,885,9,NURSE PRACTITIONER III,<div>Under medical supervision of a physician ...,Under medical supervision of a physician and w...,Nurse Practitioners
650,888,9,NURSE PRACTITIONER I - PER DIEM,Under medical supervision of a physician and w...,Under medical supervision of a physician and w...,Nurse Practitioners


#### 1. performs a left-join for jobs from same employer

In [ ]:
pair_master = pd.merge(data2, data2, how = 'left', left_on = ['EmployerID', 'BLS_Detailed'], right_on = ['EmployerID', 'BLS_Detailed'])
# pd.merge(final, data[['ClassSpecID', 'Cluster_ID']], how = 'left', left_on = 'ClassSpecID1', right_on = 'ClassSpecID')

# #rename columns
pair_master2 = pair_master.rename(columns={'ClassSpecID_x':'ClassSpecID1','ClassSpecID_y':'ClassSpecID2','Class_Title_x':'Class_Title1','Class_Title_y':'Class_Title2','EmployerID_x':'EmployerID1','EmployerID_y':'EmployerID2',\
        'New_JD_x':'New_JD1','New_JD_y':'New_JD2', 'Job_Desc_x':'Job_Desc1','Job_Desc_y':'Job_Desc2','BLS_Detailed_x':'BLS_Detailed1','BLS_Detailed_y':'BLS_Detailed2'})

# #rearrange columns and add pair_ID
pair_master2['Pair_ID'] = pair_master2['ClassSpecID1'].astype(str) + '-' + pair_master2['ClassSpecID2'].astype(str)
col_list = ['Pair_ID','ClassSpecID1','ClassSpecID2','Class_Title1','Class_Title2','BLS_Detailed','EmployerID', 'New_JD1', 'New_JD2', 'Job_Desc1','Job_Desc2']
pair_master2 = pair_master2[col_list]
pair_master2.to_csv('/content/gdrive/My Drive/Colab Notebooks/pair-master.csv')
print(pair_master2.shape)
pair_master2.head()

(1824, 11)


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,BLS_Detailed,EmployerID,New_JD1,New_JD2,Job_Desc1,Job_Desc2
0,3243-3243,3243,3243,Assistant Caseworker II,Assistant Caseworker II,Home Health Aides,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...
1,3243-3245,3243,3245,Assistant Caseworker II,Assistant Caseworker I,Home Health Aides,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...
2,3243-1094664,3243,1094664,Assistant Caseworker II,Assistant Caseworker III,Home Health Aides,168,To assess the social service needs of elderly ...,<div>To assess the social service needs of eld...,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...
3,3245-3243,3245,3243,Assistant Caseworker I,Assistant Caseworker II,Home Health Aides,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...
4,3245-3245,3245,3245,Assistant Caseworker I,Assistant Caseworker I,Home Health Aides,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...


##### Subset only BLS_Detailed == "Nurse Practitioners"

In [ ]:
pair_master3 = pair_master2[pair_master2['BLS_Detailed']=='Nurse Practitioners']
print(pair_master3.shape)
pair_master3.head()

(1104, 11)


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,BLS_Detailed,EmployerID,New_JD1,New_JD2,Job_Desc1,Job_Desc2
6,3308-3308,3308,3308,Nurse Practitioner,Nurse Practitioner,Nurse Practitioners,168,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...
7,3308-915521,3308,915521,Nurse Practitioner,Nurse Practitioner - OB,Nurse Practitioners,168,To perform physical screening assessments; to ...,<div>To perform physical screening assessments...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...
8,3308-916779,3308,916779,Nurse Practitioner,Nurse Practitioner,Nurse Practitioners,168,To perform physical screening assessments; to ...,"<div style=""text-align: justify;"">To perform p...",To perform physical screening assessments; to ...,To perform physical screening assessments; to ...
9,4916-4916,4916,4916,Family Nurse Practitioner,Family Nurse Practitioner,Nurse Practitioners,141,<strong>Bargaining Unit: CNA - Registered Nurs...,<strong>Bargaining Unit: CNA - Registered Nurs...,Bargaining Unit: CNA - Registered Nurses Unit ...,Bargaining Unit: CNA - Registered Nurses Unit ...
10,4916-959296,4916,959296,Family Nurse Practitioner,Nurse Practitioner Trainee,Nurse Practitioners,141,<strong>Bargaining Unit: CNA - Registered Nurs...,<strong>Bargaining Unit: CNA - Registered Nurs...,Bargaining Unit: CNA - Registered Nurses Unit ...,Bargaining Unit: CNA - Registered Nurses Unit ...


In [ ]:
# pair_master2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/pair-master.csv')
# print(pair_master2.shape)
# sample_titles = ['Community Health Assistant','Medical Assistant II', 'NURSING ASSISTANT','Nursing Supervisor']
    
# pair_master_5k = pair_master2[pair_master2['Class_Title1'].isin(sample_titles)]

# pair_master3 = pair_master2.copy()

pair_master3['key'] = pair_master3.apply(lambda row: tuple(sorted((row['ClassSpecID1'], row['ClassSpecID2']))), axis=1)
pair_master3.drop_duplicates(subset=['key'], keep='first', inplace=True, ignore_index=False)
print(pair_master3.shape)
pair_master3.head()

(686, 12)


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,BLS_Detailed,EmployerID,New_JD1,New_JD2,Job_Desc1,Job_Desc2,key
6,3308-3308,3308,3308,Nurse Practitioner,Nurse Practitioner,Nurse Practitioners,168,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,"(3308, 3308)"
7,3308-915521,3308,915521,Nurse Practitioner,Nurse Practitioner - OB,Nurse Practitioners,168,To perform physical screening assessments; to ...,<div>To perform physical screening assessments...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,"(3308, 915521)"
8,3308-916779,3308,916779,Nurse Practitioner,Nurse Practitioner,Nurse Practitioners,168,To perform physical screening assessments; to ...,"<div style=""text-align: justify;"">To perform p...",To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,"(3308, 916779)"
9,4916-4916,4916,4916,Family Nurse Practitioner,Family Nurse Practitioner,Nurse Practitioners,141,<strong>Bargaining Unit: CNA - Registered Nurs...,<strong>Bargaining Unit: CNA - Registered Nurs...,Bargaining Unit: CNA - Registered Nurses Unit ...,Bargaining Unit: CNA - Registered Nurses Unit ...,"(4916, 4916)"
10,4916-959296,4916,959296,Family Nurse Practitioner,Nurse Practitioner Trainee,Nurse Practitioners,141,<strong>Bargaining Unit: CNA - Registered Nurs...,<strong>Bargaining Unit: CNA - Registered Nurs...,Bargaining Unit: CNA - Registered Nurses Unit ...,Bargaining Unit: CNA - Registered Nurses Unit ...,"(4916, 959296)"


In [ ]:
GT1 = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Seniority_similarity_GT_1.xlsx')
GT1["ID"] = 1
GT1.head()

,Pair_ID,ClassSpecID_x,EmployerID_x,Class_Title_x,JD_w_senior_sentences_only_x,ClassSpecID_y,EmployerID_y,Class_Title_y,JD_w_senior_sentences_only_y,USE_sim,Human Seniority SimilaritY Score,human_seniority_sim_scr2,sbert_v1,ID
0,63345-63345,63345,1257,Nurse Practitioner I,Graduation from an accredited school of nursin...,63345,1257,Nurse Practitioner I,Graduation from an accredited school of nursin...,100.00,1,1,1.00,1
1,63345-63346,63345,1257,Nurse Practitioner I,Graduation from an accredited school of nursin...,63346,1257,Nurse Practitioner II,Serves as a role model to nursing staff and st...,83.11,0,-1,-0.20,1
2,63345-1370061,63345,1257,Nurse Practitioner I,Graduation from an accredited school of nursin...,1370061,1093,Community Health Nurse 1G,This is entry level professional staff work pe...,55.99,1,1,0.17,1
3,63345-1370062,63345,1257,Nurse Practitioner I,Graduation from an accredited school of nursin...,1370062,1093,Community Health Nurse 2G,This is full performance level professional st...,64.31,0,-1,-0.27,1
4,63346-63345,63346,1257,Nurse Practitioner II,Serves as a role model to nursing staff and st...,63345,1257,Nurse Practitioner I,Graduation from an accredited school of nursin...,83.11,0,-1,-0.20,1


In [ ]:
GT1.columns

Index(['Pair_ID', 'ClassSpecID_x', 'EmployerID_x', 'Class_Title_x',
       'JD_w_senior_sentences_only_x', 'ClassSpecID_y', 'EmployerID_y',
       'Class_Title_y', 'JD_w_senior_sentences_only_y', 'USE_sim',
       'Human Seniority SimilaritY Score', 'human_seniority_sim_scr2',
       'sbert_v1', 'ID'],
      dtype='object')

In [ ]:
#performs a left-join and gets ID column from GT1
pair_master3 = pd.merge(pair_master3, GT1[['Pair_ID','ID']], how = 'left', left_on = ['Pair_ID'], right_on = ['Pair_ID'])
pair_master3.head()

,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,BLS_Detailed,EmployerID,New_JD1,New_JD2,Job_Desc1,Job_Desc2,key,ID
0,3308-3308,3308,3308,Nurse Practitioner,Nurse Practitioner,Nurse Practitioners,168,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,"(3308, 3308)",NaN
1,3308-915521,3308,915521,Nurse Practitioner,Nurse Practitioner - OB,Nurse Practitioners,168,To perform physical screening assessments; to ...,<div>To perform physical screening assessments...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,"(3308, 915521)",NaN
2,3308-916779,3308,916779,Nurse Practitioner,Nurse Practitioner,Nurse Practitioners,168,To perform physical screening assessments; to ...,"<div style=""text-align: justify;"">To perform p...",To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,"(3308, 916779)",NaN
3,4916-4916,4916,4916,Family Nurse Practitioner,Family Nurse Practitioner,Nurse Practitioners,141,<strong>Bargaining Unit: CNA - Registered Nurs...,<strong>Bargaining Unit: CNA - Registered Nurs...,Bargaining Unit: CNA - Registered Nurses Unit ...,Bargaining Unit: CNA - Registered Nurses Unit ...,"(4916, 4916)",NaN
4,4916-959296,4916,959296,Family Nurse Practitioner,Nurse Practitioner Trainee,Nurse Practitioners,141,<strong>Bargaining Unit: CNA - Registered Nurs...,<strong>Bargaining Unit: CNA - Registered Nurs...,Bargaining Unit: CNA - Registered Nurses Unit ...,Bargaining Unit: CNA - Registered Nurses Unit ...,"(4916, 959296)",NaN


In [ ]:
pair_master3.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/pair_master3.csv',index=False)

#### 2. performs a left-join for jobs from all employers

In [ ]:
pair_master4 = pd.merge(data2, data2, how = 'left', left_on = ['BLS_Detailed'], right_on = ['BLS_Detailed'])
# pd.merge(final, data[['ClassSpecID', 'Cluster_ID']], how = 'left', left_on = 'ClassSpecID1', right_on = 'ClassSpecID')

# #rename columns
pair_master4 = pair_master4.rename(columns={'ClassSpecID_x':'ClassSpecID1','ClassSpecID_y':'ClassSpecID2','Class_Title_x':'Class_Title1','Class_Title_y':'Class_Title2','EmployerID_x':'EmployerID1','EmployerID_y':'EmployerID2',\
        'New_JD_x':'New_JD1','New_JD_y':'New_JD2', 'Job_Desc_x':'Job_Desc1','Job_Desc_y':'Job_Desc2','BLS_Detailed_x':'BLS_Detailed1','BLS_Detailed_y':'BLS_Detailed2'})

# #rearrange columns and add pair_ID
pair_master4['Pair_ID'] = pair_master4['ClassSpecID1'].astype(str) + '-' + pair_master4['ClassSpecID2'].astype(str)
col_list = ['Pair_ID','ClassSpecID1','ClassSpecID2','Class_Title1','Class_Title2','BLS_Detailed','EmployerID1','EmployerID', 'New_JD1', 'New_JD2', 'Job_Desc1','Job_Desc2']
pair_master4 = pair_master4[col_list]
pair_master4.to_csv('/content/gdrive/My Drive/Colab Notebooks/pair-master4.csv')
print(pair_master4.shape)
pair_master4.head()

(404496, 12)


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,BLS_Detailed,EmployerID1,EmployerID2,New_JD1,New_JD2,Job_Desc1,Job_Desc2
0,793-793,793,793,MEDICAL CONSULTANT,MEDICAL CONSULTANT,Nurse Practitioners,9,9,"Under direction, to provide services for vario...","Under direction, to provide services for vario...","Under direction, to provide services for vario...","Under direction, to provide services for vario..."
1,793-883,793,883,MEDICAL CONSULTANT,NURSE PRACTITIONER I,Nurse Practitioners,9,9,"Under direction, to provide services for vario...","<div style=""text-align: justify;"">Under medica...","Under direction, to provide services for vario...",Under medical supervision of a physician and w...
2,793-884,793,884,MEDICAL CONSULTANT,NURSE PRACTITIONER II,Nurse Practitioners,9,9,"Under direction, to provide services for vario...",<div>Under medical supervision of a physician ...,"Under direction, to provide services for vario...",Under medical supervision of a physician and w...
3,793-885,793,885,MEDICAL CONSULTANT,NURSE PRACTITIONER III,Nurse Practitioners,9,9,"Under direction, to provide services for vario...",<div>Under medical supervision of a physician ...,"Under direction, to provide services for vario...",Under medical supervision of a physician and w...
4,793-888,793,888,MEDICAL CONSULTANT,NURSE PRACTITIONER I - PER DIEM,Nurse Practitioners,9,9,"Under direction, to provide services for vario...",Under medical supervision of a physician and w...,"Under direction, to provide services for vario...",Under medical supervision of a physician and w...


##### Dataset-1: Diff employer but same seniority jobs

In [ ]:
Diff_emp_same_sen = pair_master4[(pair_master4['EmployerID1']!=pair_master4['EmployerID2']) & (pair_master4['Class_Title1']==pair_master4['Class_Title2'])]
Diff_emp_same_sen = Diff_emp_same_sen.sample(50)
Diff_emp_same_sen.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Diff_emp_same_sen.csv', index=False)
Diff_emp_same_sen.head()

NameError: ignored

##### Dataset-2: Same employer but different seniority jobs

In [ ]:
Same_emp_diff_sen = pair_master4[(pair_master4['EmployerID1']==pair_master4['EmployerID2']) & (pair_master4['Class_Title1']!=pair_master4['Class_Title2'])]
Same_emp_diff_sen = Same_emp_diff_sen.sample(50)
Same_emp_diff_sen.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Same_emp_diff_sen.csv', index=False)
Same_emp_diff_sen.head()

,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,BLS_Detailed,EmployerID1,EmployerID2,New_JD1,New_JD2,Job_Desc1,Job_Desc2
331791,1289197-1167004,1289197,1167004,Fixed-term Faculty Nurse Anesthetist Program,NURSING FACULTY (Family Nurse Practioner),Nurse Practitioners,2336,2336,"Metropolitan State University, Minnesota's pub...","METROPOLITAN STATE UNIVERSITY, a vibrant and g...","Metropolitan State University, Minnesota's pub...","METROPOLITAN STATE UNIVERSITY, a vibrant and g..."
99518,772074-996094,772074,996094,Psychiatric Nurse Practitioner,Nurse Practitioner,Nurse Practitioners,1470,1470,"<div align=""left"">This position assures that C...",<div>The purpose of this position is to serve ...,This position assures that CHS can fulfill its...,The purpose of this position is to serve as a ...
315752,1250710-979903,1250710,979903,Classified Staff - Grounds & Nursery Specialist 2,Grounds & Nursery Services Specialist 1 (Tempo...,Nurse Practitioners,2935,2935,Shoreline Community College is dedicated to in...,<div>This position is responsible for performi...,Shoreline Community College is dedicated to in...,This position is responsible for performing ro...
230609,1069647-1093833,1069647,1093833,Mid-level Practitioner (Physician Assistant/Fa...,"Primary Care Provider (FNP, PA)",Nurse Practitioners,1873,1873,"&nbsp;PRIMARY DUTY:&nbsp;<span style=""font-siz...",PRIMARY DUTY: The Primary Care Provider will f...,.PRIMARY DUTY:.The Mid-level Provider will fun...,PRIMARY DUTY: The Primary Care Provider will f...
198622,1005362-816703,1005362,816703,"Primary Care Provider (FNP, PA), Wellness Serv...","Family Nurse Practitioner, Wellness Services",Nurse Practitioners,1873,1873,PRIMARY DUTY: The Primary Care Provider will f...,POSITION SUMMARY: The Family Nurse Practitione...,PRIMARY DUTY: The Primary Care Provider will f...,POSITION SUMMARY: The Family Nurse Practitione...


## Text Differences

#### Method-I (ignore this)

In [ ]:
# def show_diff(seqm):
#     # function from https://stackoverflow.com/questions/774316/python-difflib-highlighting-differences-inline
#     """Unify operations between two compared strings
# seqm is a difflib.SequenceMatcher instance whose a & b are strings"""
#     output= []
#     for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
#         if opcode == 'equal':
#             output.append(seqm.a[a0:a1])
#         elif opcode == 'insert':
#             output.append("<ins>" + seqm.b[b0:b1] + "</ins>")
#         elif opcode == 'delete':
#             output.append("<del>" + seqm.a[a0:a1] + "</del>")
#         elif opcode == 'replace':
#             output.append("<ins>" + seqm.b[b0:b1] + "</ins>")
#             output.append("<del>" + seqm.a[a0:a1] + "</del>")
#         else:
#             raise RuntimeError( f"unexpected opcode unknown opcode {opcode}" )
#     return ''.join(output)

In [ ]:
# for index, row in pair_master.iterrows():
#     new_row = copy.deepcopy(row)

#     diff_text = difflib.SequenceMatcher(None, new_row['Job_Desc1'], new_row['Job_Desc2'])

#     html = '<!doctype html> <html lang="en"> <head> <title>GT-2 UI</title> <meta charset="utf-8"> <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">'+"<link href='https://fonts.googleapis.com/css?family=Roboto:400,100,300,700' rel='stylesheet' type='text/css'> " + '<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"> <link rel="stylesheet" type="text/css" href="{{ url_for('+"'static', filename='/content/gdrive/My Drive/Colab Notebooks/HTML_files/static/css/style.css'"+') }}"> </head> <style> table, th, td { border: 1px solid black; border-collapse: collapse; }, td:nth-child(even), th:nth-child(even) {background-color: #D6EEEE;} </style> <body> <section class="ftco-section"> <div class="container"> <div class="row justify-content-center"> <div class="col-md-6 text-center mb-5"> <h1 class="heading-section">GT-2: Text Diff UI</h1> </div> </div> <div class="row"> <div class="col-md-12"> <! <h4 class="text-center mb-4"> </h4> <div class="table-wrap"> <table class="table" style="height: 275px; width: 100%" border=1 frame=hsides rules=rows> <colgroup> <col span="1" style="width: 5%;"> <col span="1" style="width: 5%;"> <col span="1" style="width: 30%;"> <col span="1" style="width: 30%;"> <col span="1" style="width: 30%;"> </colgroup> <thead class="thead-primary"> <tr style="width: 100%; vertical-align:top"> <th>Class Title-1</th> <th>Class Title-2</th> <th>Job Desc-1</th> <th>Job Desc-2</th> <th> Text Difference </th> </tr> </thead> <tbody> <tr style="vertical-align:top">  <td>' + str(new_row['Class_Title1']) + '</td> <td>' + str(new_row['Class_Title2'])  + '</td> <td>'+ str(new_row['Job_Desc1'])  + '</td> <td>' + str(new_row['Job_Desc2']) + '</td> <td>' + show_diff(diff_text)  +'</td>  </tr> </tbody> </table> </div> </div> </div> </div> </section> <script src="js/jquery.min.js"></script> <script src="js/popper.js"></script> <script src="js/bootstrap.min.js"></script> <script src="js/main.js"></script> </body> </html>'

#     with open(str(new_row['ClassSpecID1'])+ '-' + str(new_row['ClassSpecID2'])+ '_diff.html', 'w') as f: 
#       f.write(html)

#### Method-II: Using difflib module to create diff indicators

In [ ]:
pair_master3 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/pair_master3.csv')
# pair_master3 = pair_master3.head(100)
print(pair_master3.shape)
pair_master3.head()

(686, 13)


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,BLS_Detailed,EmployerID,New_JD1,New_JD2,Job_Desc1,Job_Desc2,key,ID
0,3308-3308,3308,3308,Nurse Practitioner,Nurse Practitioner,Nurse Practitioners,168,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,"(3308, 3308)",NaN
1,3308-915521,3308,915521,Nurse Practitioner,Nurse Practitioner - OB,Nurse Practitioners,168,To perform physical screening assessments; to ...,<div>To perform physical screening assessments...,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,"(3308, 915521)",NaN
2,3308-916779,3308,916779,Nurse Practitioner,Nurse Practitioner,Nurse Practitioners,168,To perform physical screening assessments; to ...,"<div style=""text-align: justify;"">To perform p...",To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,"(3308, 916779)",NaN
3,4916-4916,4916,4916,Family Nurse Practitioner,Family Nurse Practitioner,Nurse Practitioners,141,<strong>Bargaining Unit: CNA - Registered Nurs...,<strong>Bargaining Unit: CNA - Registered Nurs...,Bargaining Unit: CNA - Registered Nurses Unit ...,Bargaining Unit: CNA - Registered Nurses Unit ...,"(4916, 4916)",NaN
4,4916-959296,4916,959296,Family Nurse Practitioner,Nurse Practitioner Trainee,Nurse Practitioners,141,<strong>Bargaining Unit: CNA - Registered Nurs...,<strong>Bargaining Unit: CNA - Registered Nurs...,Bargaining Unit: CNA - Registered Nurses Unit ...,Bargaining Unit: CNA - Registered Nurses Unit ...,"(4916, 959296)",NaN


In [ ]:
# pair_master3 = pair_master3[pair_master3['Pair_ID']=='76219-76227'] #4916-95929
# pair_master_final.shape

In [ ]:
# Function that matches sentences 
#https://stackoverflow.com/questions/65702508/python-difflib-get-close-matches-comparing-modified-text-but-returning-original)
from difflib import SequenceMatcher, _nlargest  # necessary imports of functions used by modified get_close_matches

def get_close_matches_lower(word, possibilities, n=1, cutoff=0.70): 
  '''word: sentence to be matched from JD1(string)
     possibilities: list of sentences to be matched with in JD2
     n: sorts sentences on the basis of match accuracy and gives 1 result
     cutoff: match accuracy
  '''
  if not n >  0:
      raise ValueError("n must be > 0: %r" % (n,))
  if not 0.0 <= cutoff <= 1.0:
      raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
  result = []
  s = SequenceMatcher()
  s.set_seq2(word)
  for x in possibilities:
      s.set_seq1(x.lower())  # lower-case for comparison
      if s.real_quick_ratio() >= cutoff and \
          s.quick_ratio() >= cutoff and \
          s.ratio() >= cutoff:
          result.append((possibilities.index(x)+1, x))

  # Move the best scorers to head of list
  result = _nlargest(n, result)
  # Strip scores for the best n matches
  return result
  #[x for sent_id, x in result]

for index, row in pair_master3.iterrows():
  new_row = copy.deepcopy(row)
  
  #split the paragraph to sentences, and delete sentences of length<30 because no english sentence usually is less than that.
  from_lines = [x.strip() for x in re.split(r'[;:.]', new_row['Job_Desc1']) if len(x)>30]  
  to_lines = [x.strip() for x in re.split(r'[;:.]', new_row['Job_Desc2'])if len(x)>30]

  #remove duplicates from the above list
  from_lines = [*set(from_lines)]
  to_lines = [*set(to_lines)]

  # print([get_close_matches_lower(x, to_lines) for x in from_lines])

  # Part-1: First round of matching (Get the sentences in JD-1 that match with JD-2)
  try:
    sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = zip(*[(from_lines.index(x)+1, x, 0, get_close_matches_lower(x, to_lines)[0][0], get_close_matches_lower(x, to_lines)[0][1], 0) for x in from_lines if len(get_close_matches_lower(x, to_lines))>0])
  except:
    sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

  # print(len(from_lines1))
  # print(len(to_lines1))
  # print(len(sent_ID1))
  # print(len(sent_ID2))

  # Create a new list by removing the matched sentences from the full list
  from_lines2 = [x for x in from_lines if x not in from_lines1]
  to_lines2 = [x for x in to_lines if x not in to_lines1]

  # print(len(from_lines2))
  # print(len(to_lines2))

  # Part-2: Second round of matching (Get the sentences in JD-2 that were not matched with JD-1 in the first round and try matching)
  try:
    sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = zip(*[(get_close_matches_lower(x, from_lines)[0][0], get_close_matches_lower(x, from_lines)[0][1], 0, to_lines.index(x)+1, x, 0) for x in to_lines2 if len(get_close_matches_lower(x, from_lines))>0])
  except:
    sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

  # Create a new list by removing the matched sentences from the non-matched list in the first round of matching
  from_lines4 = [x for x in from_lines2 if x not in from_lines3]
  sent_ID5 = [from_lines.index(x)+1 for x in from_lines4]
  to_lines4 = [x for x in to_lines2 if x not in to_lines3]
  sent_ID6 = [to_lines.index(x)+1 for x in to_lines4]

  # convert above lists into a pandas dataframe|
  lines = pd.DataFrame(list(zip(from_lines, to_lines)))
  data1 = pd.DataFrame(list(zip(sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
  data2 = pd.DataFrame(list(zip(sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
  data3 = pd.DataFrame({'sent_ID1': pd.Series(sent_ID5),'JD1': pd.Series(from_lines4), 'senior_ID1': pd.Series(['1']*len(from_lines4)), 'sent_ID2': pd.Series(['']*len(from_lines4)), 'JD2': pd.Series(['']*len(from_lines4)), 'senior_ID2': pd.Series(['']*len(from_lines4))})
  data4 = pd.DataFrame({'sent_ID1': pd.Series(['']*len(to_lines4)), 'JD1': pd.Series(['']*len(to_lines4)), 'senior_ID1': pd.Series(['']*len(to_lines4)), 'sent_ID2': pd.Series(sent_ID6), 'JD2': pd.Series(to_lines4), 'senior_ID2': pd.Series(['1']*len(to_lines4))})

  # append the above 4 datasets
  data = data1.append(data2)
  data.sort_values('sent_ID1')
  data = data.append(data3)
  data = data.append(data4)

  data = data.rename(columns={'JD1': str(new_row['Class_Title1']), 'JD2': str(new_row['Class_Title2']), 'senior_ID1': 'Diff_text1', 'senior_ID2': 'Diff_text2'})
  # data.head()
  data.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Diff text by Pair_ID/'+str(new_row['Pair_ID'])+"_sentences.csv", index=False)
  # data1.to_csv('test1.csv', index=False)
  # data2.to_csv('test2.csv', index=False)
  # lines.to_csv('lines.csv', index=False)

#### Difftext for Diff employer but same seniority jobs

In [ ]:
# Function that matches sentences 
#https://stackoverflow.com/questions/65702508/python-difflib-get-close-matches-comparing-modified-text-but-returning-original)
from difflib import SequenceMatcher, _nlargest  # necessary imports of functions used by modified get_close_matches

def get_close_matches_lower(word, possibilities, n=1, cutoff=0.70): 
  '''word: sentence to be matched from JD1(string)
     possibilities: list of sentences to be matched with in JD2
     n: sorts sentences on the basis of match accuracy and gives 1 result
     cutoff: match accuracy
  '''
  if not n >  0:
      raise ValueError("n must be > 0: %r" % (n,))
  if not 0.0 <= cutoff <= 1.0:
      raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
  result = []
  s = SequenceMatcher()
  s.set_seq2(word)
  for x in possibilities:
      s.set_seq1(x.lower())  # lower-case for comparison
      if s.real_quick_ratio() >= cutoff and \
          s.quick_ratio() >= cutoff and \
          s.ratio() >= cutoff:
          result.append((possibilities.index(x)+1, x))

  # Move the best scorers to head of list
  result = _nlargest(n, result)
  # Strip scores for the best n matches
  return result
  #[x for sent_id, x in result]

for index, row in Diff_emp_same_sen.iterrows():
  new_row = copy.deepcopy(row)
  
  #split the paragraph to sentences, and delete sentences of length<30 because no english sentence usually is less than that.
  from_lines = [x.strip() for x in re.split(r'[;:.]', new_row['Job_Desc1']) if len(x)>30]  
  to_lines = [x.strip() for x in re.split(r'[;:.]', new_row['Job_Desc2'])if len(x)>30]

  #remove duplicates from the above list
  from_lines = [*set(from_lines)]
  to_lines = [*set(to_lines)]

  # print([get_close_matches_lower(x, to_lines) for x in from_lines])

  # Part-1: First round of matching (Get the sentences in JD-1 that match with JD-2)
  try:
    sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = zip(*[(from_lines.index(x)+1, x, 0, get_close_matches_lower(x, to_lines)[0][0], get_close_matches_lower(x, to_lines)[0][1], 0) for x in from_lines if len(get_close_matches_lower(x, to_lines))>0])
  except:
    sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

  # print(len(from_lines1))
  # print(len(to_lines1))
  # print(len(sent_ID1))
  # print(len(sent_ID2))

  # Create a new list by removing the matched sentences from the full list
  from_lines2 = [x for x in from_lines if x not in from_lines1]
  to_lines2 = [x for x in to_lines if x not in to_lines1]

  # print(len(from_lines2))
  # print(len(to_lines2))

  # Part-2: Second round of matching (Get the sentences in JD-2 that were not matched with JD-1 in the first round and try matching)
  try:
    sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = zip(*[(get_close_matches_lower(x, from_lines)[0][0], get_close_matches_lower(x, from_lines)[0][1], 0, to_lines.index(x)+1, x, 0) for x in to_lines2 if len(get_close_matches_lower(x, from_lines))>0])
  except:
    sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

  # Create a new list by removing the matched sentences from the non-matched list in the first round of matching
  from_lines4 = [x for x in from_lines2 if x not in from_lines3]
  sent_ID5 = [from_lines.index(x)+1 for x in from_lines4]
  to_lines4 = [x for x in to_lines2 if x not in to_lines3]
  sent_ID6 = [to_lines.index(x)+1 for x in to_lines4]

  # convert above lists into a pandas dataframe|
  lines = pd.DataFrame(list(zip(from_lines, to_lines)))
  data1 = pd.DataFrame(list(zip(sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
  data2 = pd.DataFrame(list(zip(sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
  data3 = pd.DataFrame({'sent_ID1': pd.Series(sent_ID5),'JD1': pd.Series(from_lines4), 'senior_ID1': pd.Series(['1']*len(from_lines4)), 'sent_ID2': pd.Series(['']*len(from_lines4)), 'JD2': pd.Series(['']*len(from_lines4)), 'senior_ID2': pd.Series(['']*len(from_lines4))})
  data4 = pd.DataFrame({'sent_ID1': pd.Series(['']*len(to_lines4)), 'JD1': pd.Series(['']*len(to_lines4)), 'senior_ID1': pd.Series(['']*len(to_lines4)), 'sent_ID2': pd.Series(sent_ID6), 'JD2': pd.Series(to_lines4), 'senior_ID2': pd.Series(['1']*len(to_lines4))})

  # append the above 4 datasets
  data = data1.append(data2)
  data.sort_values('sent_ID1')
  data = data.append(data3)
  data = data.append(data4)

  data = data.rename(columns={'JD1': str(new_row['Class_Title1']), 'JD2': str(new_row['Class_Title2']), 'senior_ID1': 'Diff_text1', 'senior_ID2': 'Diff_text2'})
  # data.head()
  data.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Diff text by Pair_ID/Diff_emp_same_sen/'+str(new_row['Pair_ID'])+"_sentences.csv", index=False)
  # data1.to_csv('test1.csv', index=False)
  # data2.to_csv('test2.csv', index=False)
  # lines.to_csv('lines.csv', index=False)

#### Difftext for Same employer but different seniority jobs

In [ ]:
# Function that matches sentences 
#https://stackoverflow.com/questions/65702508/python-difflib-get-close-matches-comparing-modified-text-but-returning-original)
from difflib import SequenceMatcher, _nlargest  # necessary imports of functions used by modified get_close_matches

def get_close_matches_lower(word, possibilities, n=1, cutoff=0.70): 
  '''word: sentence to be matched from JD1(string)
     possibilities: list of sentences to be matched with in JD2
     n: sorts sentences on the basis of match accuracy and gives 1 result
     cutoff: match accuracy
  '''
  if not n >  0:
      raise ValueError("n must be > 0: %r" % (n,))
  if not 0.0 <= cutoff <= 1.0:
      raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
  result = []
  s = SequenceMatcher()
  s.set_seq2(word)
  for x in possibilities:
      s.set_seq1(x.lower())  # lower-case for comparison
      if s.real_quick_ratio() >= cutoff and \
          s.quick_ratio() >= cutoff and \
          s.ratio() >= cutoff:
          result.append((possibilities.index(x)+1, x))

  # Move the best scorers to head of list
  result = _nlargest(n, result)
  # Strip scores for the best n matches
  return result
  #[x for sent_id, x in result]

for index, row in Same_emp_diff_sen.iterrows():
  new_row = copy.deepcopy(row)
  
  #split the paragraph to sentences, and delete sentences of length<30 because no english sentence usually is less than that.
  from_lines = [x.strip() for x in re.split(r'[;:.]', new_row['Job_Desc1']) if len(x)>30]  
  to_lines = [x.strip() for x in re.split(r'[;:.]', new_row['Job_Desc2'])if len(x)>30]

  #remove duplicates from the above list
  from_lines = [*set(from_lines)]
  to_lines = [*set(to_lines)]

  # print([get_close_matches_lower(x, to_lines) for x in from_lines])

  # Part-1: First round of matching (Get the sentences in JD-1 that match with JD-2)
  try:
    sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = zip(*[(from_lines.index(x)+1, x, 0, get_close_matches_lower(x, to_lines)[0][0], get_close_matches_lower(x, to_lines)[0][1], 0) for x in from_lines if len(get_close_matches_lower(x, to_lines))>0])
  except:
    sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

  # print(len(from_lines1))
  # print(len(to_lines1))
  # print(len(sent_ID1))
  # print(len(sent_ID2))

  # Create a new list by removing the matched sentences from the full list
  from_lines2 = [x for x in from_lines if x not in from_lines1]
  to_lines2 = [x for x in to_lines if x not in to_lines1]

  # print(len(from_lines2))
  # print(len(to_lines2))

  # Part-2: Second round of matching (Get the sentences in JD-2 that were not matched with JD-1 in the first round and try matching)
  try:
    sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = zip(*[(get_close_matches_lower(x, from_lines)[0][0], get_close_matches_lower(x, from_lines)[0][1], 0, to_lines.index(x)+1, x, 0) for x in to_lines2 if len(get_close_matches_lower(x, from_lines))>0])
  except:
    sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

  # Create a new list by removing the matched sentences from the non-matched list in the first round of matching
  from_lines4 = [x for x in from_lines2 if x not in from_lines3]
  sent_ID5 = [from_lines.index(x)+1 for x in from_lines4]
  to_lines4 = [x for x in to_lines2 if x not in to_lines3]
  sent_ID6 = [to_lines.index(x)+1 for x in to_lines4]

  # convert above lists into a pandas dataframe|
  lines = pd.DataFrame(list(zip(from_lines, to_lines)))
  data1 = pd.DataFrame(list(zip(sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
  data2 = pd.DataFrame(list(zip(sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
  data3 = pd.DataFrame({'sent_ID1': pd.Series(sent_ID5),'JD1': pd.Series(from_lines4), 'senior_ID1': pd.Series(['1']*len(from_lines4)), 'sent_ID2': pd.Series(['']*len(from_lines4)), 'JD2': pd.Series(['']*len(from_lines4)), 'senior_ID2': pd.Series(['']*len(from_lines4))})
  data4 = pd.DataFrame({'sent_ID1': pd.Series(['']*len(to_lines4)), 'JD1': pd.Series(['']*len(to_lines4)), 'senior_ID1': pd.Series(['']*len(to_lines4)), 'sent_ID2': pd.Series(sent_ID6), 'JD2': pd.Series(to_lines4), 'senior_ID2': pd.Series(['1']*len(to_lines4))})

  # append the above 4 datasets
  data = data1.append(data2)
  data.sort_values('sent_ID1')
  data = data.append(data3)
  data = data.append(data4)

  data = data.rename(columns={'JD1': str(new_row['Class_Title1']), 'JD2': str(new_row['Class_Title2']), 'senior_ID1': 'Diff_text1', 'senior_ID2': 'Diff_text2'})
  # data.head()
  data.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Diff text by Pair_ID/Same_emp_diff_sen/'+str(new_row['Pair_ID'])+"_sentences.csv", index=False)
  # data1.to_csv('test1.csv', index=False)
  # data2.to_csv('test2.csv', index=False)
  # lines.to_csv('lines.csv', index=False)

#### Create diff indicators (Use Darish's code to split sentences)

In [ ]:
# pair_master_final = pair_master3.copy()
# # pair_master_final = pair_master3[pair_master3['Pair_ID']=='3308-915521']
# # pair_master3 = pair_master[~pair_master['Pair_ID'].isin(['1094664-3243'])]
# # print(pair_master3.shape)


# # Function that matches sentences 
# #https://stackoverflow.com/questions/65702508/python-difflib-get-close-matches-comparing-modified-text-but-returning-original)
# from difflib import SequenceMatcher, _nlargest  # necessary imports of functions used by modified get_close_matches

# def get_close_matches_lower(word, possibilities, n=1, cutoff=0.70): 
#   '''word: sentence to be matched from JD1(string)
#      possibilities: list of sentences to be matched with in JD2
#      n: sorts sentences on the basis of match accuracy and gives 1 result
#      cutoff: match accuracy
#   '''
#   if not n >  0:
#       raise ValueError("n must be > 0: %r" % (n,))
#   if not 0.0 <= cutoff <= 1.0:
#       raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
#   result = []
#   s = SequenceMatcher()
#   s.set_seq2(word)
#   for x in possibilities:
#       s.set_seq1(x.lower())  # lower-case for comparison
#       if s.real_quick_ratio() >= cutoff and \
#           s.quick_ratio() >= cutoff and \
#           s.ratio() >= cutoff:
#           result.append((possibilities.index(x)+1, x))

#   # Move the best scorers to head of list
#   result = _nlargest(n, result)
#   # Strip scores for the best n matches
#   return result
#   #[x for sent_id, x in result]

# for index, row in pair_master_final.iterrows():
#   new_row = copy.deepcopy(row)
  
#   new_row['Job_Desc1'] = new_row['Job_Desc1'].replace(';','.').replace(':','.')
#   new_row['Job_Desc2'] = new_row['Job_Desc2'].replace(';','.').replace(':','.')

#   #split the paragraph to sentences, and delete sentences of length<30 because no english sentence usually is less than that.
#   from_lines = [x.strip() for x in re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', new_row['Job_Desc1']) if len(x)>30]  
#   to_lines = [x.strip() for x in re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', new_row['Job_Desc2'])if len(x)>30]

#   # print([get_close_matches_lower(x, to_lines) for x in from_lines])

#   # Part-1: First round of matching (Get the sentences in JD-1 that match with JD-2)
#   try:
#     sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = zip(*[(from_lines.index(x)+1, x, 0, get_close_matches_lower(x, to_lines)[0][0], get_close_matches_lower(x, to_lines)[0][1], 0) for x in from_lines if len(get_close_matches_lower(x, to_lines))>0])
#   except:
#     sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

#   # print(len(from_lines1))
#   # print(len(to_lines1))
#   # print(len(sent_ID1))
#   # print(len(sent_ID2))

#   # Create a new list by removing the matched sentences from the full list
#   from_lines2 = [x for x in from_lines if x not in from_lines1]
#   to_lines2 = [x for x in to_lines if x not in to_lines1]

#   # print(len(from_lines2))
#   # print(len(to_lines2))

#   # Part-2: Second round of matching (Get the sentences in JD-2 that were not matched with JD-1 in the first round and try matching)
#   try:
#     sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = zip(*[(to_lines.index(x)+1, x, 0, get_close_matches_lower(x, from_lines)[0][0], get_close_matches_lower(x, from_lines)[0][1], 0) for x in to_lines2 if len(get_close_matches_lower(x, from_lines))>0])
#   except:
#     sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

#   # Create a new list by removing the matched sentences from the non-matched list in the first round of matching
#   from_lines4 = [x for x in from_lines2 if x not in from_lines3]
#   sent_ID5 = [from_lines.index(x)+1 for x in from_lines4]
#   to_lines4 = [x for x in to_lines2 if x not in to_lines3]
#   sent_ID6 = [to_lines.index(x)+1 for x in to_lines4]

#   # convert above lists into a pandas dataframe|
#   data1 = pd.DataFrame(list(zip(sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
#   data2 = pd.DataFrame(list(zip(sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
#   data3 = pd.DataFrame({'sent_ID1': pd.Series(sent_ID5),'JD1': pd.Series(from_lines4), 'senior_ID1': pd.Series(['1']*len(from_lines4)), 'sent_ID2': pd.Series(['']*len(from_lines4)), 'JD2': pd.Series(['']*len(from_lines4)), 'senior_ID2': pd.Series(['']*len(from_lines4))})
#   data4 = pd.DataFrame({'sent_ID1': pd.Series(['']*len(to_lines4)), 'JD1': pd.Series(['']*len(to_lines4)), 'senior_ID1': pd.Series(['']*len(to_lines4)), 'sent_ID2': pd.Series(sent_ID6), 'JD2': pd.Series(to_lines4), 'senior_ID2': pd.Series(['1']*len(to_lines4))})

#   # append the above 4 datasets
#   data = data1.append(data2)
#   data.sort_values('sent_ID1')
#   data = data.append(data3)
#   data = data.append(data4)

#   data = data.rename(columns={'JD1': str(new_row['Class_Title1']), 'JD2': str(new_row['Class_Title2'])})
# # data.head()
#   data.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Sentence split by Pair_ID/'+str(new_row['Pair_ID'])+"_sentences_new.csv", index=False)

In [ ]:
#   # from_lines = [f'{i}.'.strip() for i in new_row['Job_Desc1'].strip().split('.|;|:')]
#   # to_lines = [f'{i}.'.strip() for i in new_row['Job_Desc2'].strip().split('.')]

#   # print("\n\n".join(to_lines))
#   # print("\n\n".join(from_lines))

#   # from_lines = nltk.tokenize.sent_tokenize(new_row['Job_Desc1'])
#   # to_lines = nltk.tokenize.sent_tokenize(new_row['Job_Desc2'])

#   ###### Part-1: simple string match #######

#   # create multiple empty lists
#   list1, list2, listnum1, listnum2, Diff_against_job1, Diff_against_job2, Job_ID1, Job_ID2 = ([] for i in range(8))

#   for sent1 in from_lines:
#     Job_ID1.append(new_row['Class_Title1'])
#     Diff_against_job1.append(new_row['Class_Title2'])
#     listnum1.append(from_lines.index(sent1)+1)
#     if sent1 in to_lines:
#       list1.append(0)
#     else:
#       list1.append(1)

#   for sent2 in to_lines:
#     Job_ID2.append(new_row['Class_Title2'])
#     Diff_against_job2.append(new_row['Class_Title1'])
#     listnum2.append(to_lines.index(sent2)+1)
#     if sent2 in from_lines:
#       list2.append(0)
#     else:
#       list2.append(1)

#   data1 = pd.DataFrame(list(zip(Job_ID1, listnum1, from_lines, Diff_against_job1, list1)),columns =['Class_title','Sent_ID', 'Sentence', 'Diff_against_Class_Title', 'Diff_ind'])
#   data2 = pd.DataFrame(list(zip(Job_ID2, listnum2, to_lines, Diff_against_job2, list2)),columns =['Class_title','Sent_ID', 'Sentence', 'Diff_against_Class_Title', 'Diff_ind'])

#   final = data1.append(data2)
#   # final['Sentence1'] = final['Sentence'].astype("string")
#   final['Sentence'] = final['Sentence'].apply(lambda x: x.strip()) 

#   # print(len(final))
  
#   ###### Part-2: Difflib #######
#   d = difflib.HtmlDiff(wrapcolumn=80)
#   html = d.make_file(from_lines, to_lines, str(new_row['Class_Title1']), str(new_row['Class_Title2']))

#   with open('/content/gdrive/My Drive/Colab Notebooks/HTML_files/Seniority_sent/'+str(new_row['Pair_ID'])+'_diff.html', 'w') as fh: 
#     fh.write(html)

#   #Apply difflib.Differ() and clean the results to get matched sentences
#   z = difflib.Differ()
#   sent_match = [line for line in z.compare(from_lines, to_lines)]
  
#   def symbol(s):
#     return s[0].strip()

#   def sentence(s):
#     return s[1:].strip()

#   Symbol = [symbol(s) for s in sent_match]
#   Sentence = [sentence(s) for s in sent_match]

#   sent_data = pd.DataFrame(list(zip(sent_match, Symbol, Sentence)),columns =['Sent_match', 'Symbol', 'Sentence'])
#   sent_data['ID'] = 1
  
#   # Give ID (indicator) = 0 if the sentences in the JDs match, else 1
#   for i in range(len(sent_data)):
#     if sent_data['Symbol'][i]=='':
#       sent_data['ID'][i] = 0
#     if sent_data['Symbol'][i] == '?':
#       sent_data['ID'][i-1] = 0

#   ###### Part-3: Merge final and sent_data dataframes ###### 
#   sent_data['Sentence'] = sent_data['Sentence'].astype(str)
#   final['Sentence'] = final['Sentence'].astype(str)
#   final = final.merge(sent_data, on='Sentence', how='left')

#   #update Diff_ind column in final dataset based on ID column from sent_data
#   for i in range(len(final)):
#     if final['Diff_ind'][i] != final['ID'][i]:
#       final['Diff_ind'][i] = final['ID'][i]

#   final.drop(['Sent_match',	'Symbol', 'ID'], axis=1, inplace=True)
#   final.to_csv('/content/gdrive/My Drive/Colab Notebooks/HTML_files/Seniority_sent/'+str(new_row['Pair_ID'])+"_final.csv", index=False)

# # final.head()
# # # # sent_data.head()


# # #   # print('\n'.join(line for line in z.compare(from_lines, to_lines))) # if not line.startswith(' ')
# # #   # result = list(z.compare(from_lines, to_lines))
# # #   # from pprint import pprint
# # #   # pprint(result)

# # #   # open in web browser
# # #   # webbrowser.open('output.html')

In [ ]:
# for index, row in pair_master2.iterrows():
#   new_row = copy.deepcopy(row)

#   #split the paragraph to sentences
#   from_lines = [f'{i}.'.strip() for i in new_row['Job_Desc1'].strip().split('.')]
#   to_lines = [f'{i}.'.strip() for i in new_row['Job_Desc2'].strip().split('.')]

#   #remove unexpected short sentences 
#   from_lines = [i for i in from_lines if len(i) >10]
#   to_lines = [i for i in to_lines if len(i) >10]

#   # print("\n---\n".join(from_lines))

#   #create multiple empty lists
#   list1, list2, listnum1, listnum2, Diff_against_job1, Diff_against_job2, Job_ID1, Job_ID2 = ([] for i in range(8))

#   for sent1 in from_lines:
#     Job_ID1.append(new_row['Class_Title1'])
#     Diff_against_job1.append(new_row['Class_Title2'])
#     listnum1.append(from_lines.index(sent1)+1)
#     if sent1 in to_lines:
#       list1.append(0)
#     else:
#       list1.append(1)

#   for sent2 in to_lines:
#     Job_ID2.append(new_row['Class_Title2'])
#     Diff_against_job2.append(new_row['Class_Title1'])
#     listnum2.append(to_lines.index(sent2)+1)
#     if sent2 in from_lines:
#       list2.append(0)
#     else:
#       list2.append(1)

#   data1 = pd.DataFrame(list(zip(Job_ID1, listnum1, from_lines, Diff_against_job1, list1)),columns =['Class_title','Sent_ID', 'Sentence_text', 'Diff_against_Class_Title', 'Diff_ind'])
#   data2 = pd.DataFrame(list(zip(Job_ID2, listnum2, to_lines, Diff_against_job2, list2)),columns =['Class_title','Sent_ID', 'Sentence_text', 'Diff_against_Class_Title', 'Diff_ind'])

#   final = data1.append(data2)
#   final.to_csv("final.csv", index=False)

## Seniority text GT (Check for duplicates and join by Pair_ID)

In [ ]:
sen_data = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/GT2 seniority sentences_Nurse_v2_2022-07-26.xlsx', sheet_name='sentence_master3')
print(sen_data.shape)
sen_data.head()

(794, 16)


,Pair-id,ClassSpec ID,Employer ID,Class Title 1,BLS_Detail,Sent ID,JD Sentence,Diff Indicator,ClassSpec ID2,Employer ID2,Class Title 2,Sent ID2 (Optional),JD Sententce 2 (Optional),"Human Seniority Label (only label very strong seniority sent as 1, else 0)",Labeller,Reviewed
0,716751-716754,716754,1597,Nurse Practitioner II,Nurse Practitioners,11,This is a summary of the duties and responsibi...,0,716751,1597,Nurse Practitioner I,NaN,NaN,0,VR,AK
1,716751-716754,716754,1597,Nurse Practitioner II,Nurse Practitioners,12,"It is not meant to be all-inclusive, thus, oth...",0,716751,1597,Nurse Practitioner I,NaN,NaN,0,VR,AK
2,716751-716754,716754,1597,Nurse Practitioner II,Nurse Practitioners,25,Skill in analyzing and assessing clients medic...,0,716751,1597,Nurse Practitioner I,NaN,NaN,0,VR,AK
3,716751-716754,716754,1597,Nurse Practitioner II,Nurse Practitioners,26,"prescribing appropriate treatment, therapy and...",0,716751,1597,Nurse Practitioner I,NaN,NaN,1,VR,AK
4,716751-716754,716754,1597,Nurse Practitioner II,Nurse Practitioners,27,"and compiling, assessing, and charting clients...",0,716751,1597,Nurse Practitioner I,NaN,NaN,1,VR,AK


In [ ]:
# sorting by first name
sen_data.sort_values("JD Sentence", inplace = True)
 
# dropping ALL duplicate values
sen_data.drop_duplicates(subset =["ClassSpec ID", "JD Sentence"], keep = False, inplace = True)
# sen_data = sen_data.loc[sen_data.duplicated(subset =["ClassSpec ID", "JD Sentence"]), :]
print(sen_data.shape)

(767, 16)


In [ ]:
sen_data.head()

,Pair-id,ClassSpec ID,Employer ID,Class Title 1,BLS_Detail,Sent ID,JD Sentence,Diff Indicator,ClassSpec ID2,Employer ID2,Class Title 2,Sent ID2 (Optional),JD Sententce 2 (Optional),"Human Seniority Label (only label very strong seniority sent as 1, else 0)",Labeller,Reviewed
622,1250710-979903,1250710,2935,Classified Staff - Grounds & Nursery Specialist 2,Nurse Practitioners,13,"""Fostering a climate of equity and belonging t...",1,996094,1470,Nurse Practitioner,NaN,NaN,0,DS,0
625,1250710-979903,1250710,2935,Classified Staff - Grounds & Nursery Specialist 2,Nurse Practitioners,16,"""NEOGOV online application/profile",1,996094,1470,Nurse Practitioner,NaN,NaN,0,DS,0
621,1250710-979903,1250710,2935,Classified Staff - Grounds & Nursery Specialist 2,Nurse Practitioners,11,"""Operation and maintenance of power and motori...",1,996094,1470,Nurse Practitioner,NaN,NaN,0,DS,0
624,1250710-979903,1250710,2935,Classified Staff - Grounds & Nursery Specialist 2,Nurse Practitioners,15,"""Operation of hand tools and power and motoriz...",1,996094,1470,Nurse Practitioner,NaN,NaN,0,DS,0
606,1250710-979903,1250710,2935,Classified Staff - Grounds & Nursery Specialist 2,Nurse Practitioners,1,"""Valid unrestricted Washington State driver's ...",0,996094,1470,Nurse Practitioner,NaN,NaN,1,DS,1


In [ ]:
sen_data.columns

Index(['Pair-id', 'ClassSpec ID', 'Employer ID', 'Class Title 1', 'BLS_Detail',
       'Sent ID', 'JD Sentence', 'Diff Indicator', 'ClassSpec ID2',
       'Employer ID2', 'Class Title 2', 'Sent ID2 (Optional)',
       'JD Sententce 2 (Optional)',
       'Human Seniority Label (only label very strong seniority sent as 1, else 0)',
       'Labeller', 'Reviewed'],
      dtype='object')

#### Append seniority setences into one paragraph per ClassSpecID

In [ ]:
sen_data2 = sen_data[['ClassSpec ID','Employer ID', 'Class Title 1', 'BLS_Detail', 'JD Sentence', 'Human Seniority Label (only label very strong seniority sent as 1, else 0)']].copy()
sen_data2['Human Seniority Label (only label very strong seniority sent as 1, else 0)'] = sen_data2['Human Seniority Label (only label very strong seniority sent as 1, else 0)'].apply(str)
sen_data2 = sen_data2.groupby(['ClassSpec ID','Employer ID', 'Class Title 1', 'BLS_Detail', 'Human Seniority Label (only label very strong seniority sent as 1, else 0)'])['JD Sentence'].apply('. '.join)

#converts above output to a pandas dataframe
sen_data2 = sen_data2.to_frame()
sen_data2 = sen_data2.unstack(level='Human Seniority Label (only label very strong seniority sent as 1, else 0)')
sen_data2.reset_index(inplace=True)
sen_data2.columns = sen_data2.columns.map(''.join)

sen_data2 = sen_data2[['ClassSpec ID','Employer ID', 'Class Title 1', 'BLS_Detail', 'JD Sentence1']]
print(sen_data2.shape)
sen_data2.head(9)
# type(sen_data2)

(19, 5)


,ClassSpec ID,Employer ID,Class Title 1,BLS_Detail,JD Sentence1
0,885,9,NURSE PRACTITIONER III,Nurse Practitioners,A Bachelor's degree from a recognized college ...
1,3308,168,Nurse Practitioner,Nurse Practitioners,A valid Registered Nurse certificate issued by...
2,4916,141,Family Nurse Practitioner,Nurse Practitioners,"As directed, evaluates nursing services, recom..."
3,5875,385,Nurse Practitioner,Nurse Practitioners,Collaborate with physicians and other health p...
4,8102,43,CLINICAL NURSE SPECIALIST II,Nurse Practitioners,A Masters degree in a clinical field of nursin...
5,27054,761,NURSE PRACTITIONER I,Nurse Practitioners,Highest level of mental aptitude required to p...
6,63345,1257,Nurse Practitioner I,Nurse Practitioners,Graduation from an accredited school of nursin...
7,76219,421,Nurse Practitioner I,Nurse Practitioners,Candidates will be required to obtain a Drug D...
8,76227,421,Nurse Practitioner II,Nurse Practitioners,Candidates will be required to obtain a Drug D...


#### Create Pair-wise Master data again

In [ ]:
#performs a cross-join
sen_pair_master = pd.merge(sen_data2, sen_data2, how = 'left', left_on = ['BLS_Detail'], right_on = ['BLS_Detail'])
# pd.merge(final, data[['ClassSpecID', 'Cluster_ID']], how = 'left', left_on = 'ClassSpecID1', right_on = 'ClassSpecID')

# #rename columns
sen_pair_master2 = sen_pair_master.rename(columns={'ClassSpec ID_x':'ClassSpecID1','ClassSpec ID_y':'ClassSpecID2','Class Title 1_x':'Class_Title1','Class Title 1_y':'Class_Title2','Employer ID_x':'EmployerID1','Employer ID_y':'EmployerID2',\
        'JD Sentence1_x':'Sen_text1','JD Sentence1_y':'Sen_text2'})

# #rearrange columns and add pair_ID
sen_pair_master2['Pair_ID'] = sen_pair_master2['ClassSpecID1'].astype(str) + '-' + sen_pair_master2['ClassSpecID2'].astype(str)
col_list = ['Pair_ID','ClassSpecID1','ClassSpecID2','Class_Title1','Class_Title2','EmployerID1', 'EmployerID2','BLS_Detail','Sen_text1','Sen_text2']
sen_pair_master2 = sen_pair_master2[col_list]

### Drop the pairs that dont make sense
#(1) Drop pairs with same job titles
sen_pair_master2 = sen_pair_master2[~((sen_pair_master2['EmployerID1']==sen_pair_master2['EmployerID2'])&(sen_pair_master2['Class_Title1']==sen_pair_master2['Class_Title2']))]

sen_pair_master2.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Sen_pair_master2.csv', index = False)
print(sen_pair_master2.shape)
sen_pair_master2.head()

(336, 10)


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID1,EmployerID2,BLS_Detail,Sen_text1,Sen_text2
1,885-3308,885,3308,NURSE PRACTITIONER III,Nurse Practitioner,9,168,Nurse Practitioners,A Bachelor's degree from a recognized college ...,A valid Registered Nurse certificate issued by...
2,885-4916,885,4916,NURSE PRACTITIONER III,Family Nurse Practitioner,9,141,Nurse Practitioners,A Bachelor's degree from a recognized college ...,"As directed, evaluates nursing services, recom..."
3,885-5875,885,5875,NURSE PRACTITIONER III,Nurse Practitioner,9,385,Nurse Practitioners,A Bachelor's degree from a recognized college ...,Collaborate with physicians and other health p...
4,885-8102,885,8102,NURSE PRACTITIONER III,CLINICAL NURSE SPECIALIST II,9,43,Nurse Practitioners,A Bachelor's degree from a recognized college ...,A Masters degree in a clinical field of nursin...
5,885-27054,885,27054,NURSE PRACTITIONER III,NURSE PRACTITIONER I,9,761,Nurse Practitioners,A Bachelor's degree from a recognized college ...,Highest level of mental aptitude required to p...


In [ ]:
Sen_pair_final = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/GT2 seniority sentences_Nurse_v2_2022-07-26.xlsx', sheet_name='Sen_pair_master2')
# Sen_pair_final["ID"] = 1
Sen_pair_final.head()

,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID1,EmployerID2,BLS_Detail,Sen_text1,Sen_text2,Sen_Ind
0,3308-4916,3308.0,4916.0,Nurse Practitioner,Family Nurse Practitioner,168.0,141.0,Nurse Practitioners,A valid Registered Nurse certificate issued by...,"As directed, evaluates nursing services, recom...",-1.0
1,3308-63345,3308.0,63345.0,Nurse Practitioner,Nurse Practitioner I,168.0,1257.0,Nurse Practitioners,A valid Registered Nurse certificate issued by...,Graduation from an accredited school of nursin...,-1.0
2,3308-76219,3308.0,76219.0,Nurse Practitioner,Nurse Practitioner I,168.0,421.0,Nurse Practitioners,A valid Registered Nurse certificate issued by...,Candidates will be required to obtain a Drug D...,-1.0
3,3308-76227,3308.0,76227.0,Nurse Practitioner,Nurse Practitioner II,168.0,421.0,Nurse Practitioners,A valid Registered Nurse certificate issued by...,Candidates will be required to obtain a Drug D...,-1.0
4,3308-716754,3308.0,716754.0,Nurse Practitioner,Nurse Practitioner II,168.0,1597.0,Nurse Practitioners,A valid Registered Nurse certificate issued by...,An employee in this class is responsible for n...,-1.0


In [ ]:
Sen_pair_master.columns

Index(['Pair_ID', 'ClassSpecID1', 'ClassSpecID2', 'Class_Title1',
       'Class_Title2', 'EmployerID1', 'EmployerID2', 'BLS_Detail', 'Sen_text1',
       'Sen_text2', 'Unnamed: 10', 'ID'],
      dtype='object')

In [ ]:
#performs a left-join and gets ID column from GT1
sen_pair_master2 = pd.merge(sen_pair_master2, Sen_pair_final[['Pair_ID','Sen_Ind']], how = 'left', left_on = ['Pair_ID'], right_on = ['Pair_ID'])
sen_pair_master2.head()

,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID1,EmployerID2,BLS_Detail,Sen_text1,Sen_text2,Sen_Ind
0,885-3308,885,3308,NURSE PRACTITIONER III,Nurse Practitioner,9,168,Nurse Practitioners,A Bachelor's degree from a recognized college ...,A valid Registered Nurse certificate issued by...,NaN
1,885-4916,885,4916,NURSE PRACTITIONER III,Family Nurse Practitioner,9,141,Nurse Practitioners,A Bachelor's degree from a recognized college ...,"As directed, evaluates nursing services, recom...",NaN
2,885-5875,885,5875,NURSE PRACTITIONER III,Nurse Practitioner,9,385,Nurse Practitioners,A Bachelor's degree from a recognized college ...,Collaborate with physicians and other health p...,NaN
3,885-8102,885,8102,NURSE PRACTITIONER III,CLINICAL NURSE SPECIALIST II,9,43,Nurse Practitioners,A Bachelor's degree from a recognized college ...,A Masters degree in a clinical field of nursin...,NaN
4,885-27054,885,27054,NURSE PRACTITIONER III,NURSE PRACTITIONER I,9,761,Nurse Practitioners,A Bachelor's degree from a recognized college ...,Highest level of mental aptitude required to p...,NaN


In [ ]:
sen_pair_master2.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/sen_pair_master2_new.csv',index=False)